In [1]:
import json
import pandas as pd

In [2]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)

In [3]:
# make discord name
discord_username_full = user['username'] + '#' + str(user['discriminator'])

In [4]:
# read servers index file
with open("package/servers/index.json", "r") as read_file:
    servers_index = json.load(read_file)

In [5]:
# count no of guilds currently in
no_of_guilds = len(servers_index)

In [6]:
no_of_guilds

37

In [7]:
# read messages index file
with open("package/messages/index.json", "r") as read_file:
    messages_index = json.load(read_file)

In [8]:
no_of_chats = len(messages_index)

In [9]:
no_of_chats

764

In [10]:
def dm_check(key, val):
    try:
        if val[:19] == "Direct Message with":
            # is a dm
            return 0
        else:
            # is a server channel
            return 1
    except:
        # is null
        return 2

In [11]:
dms_index = dict()
server_channel_index = dict()
null_index = dict()
for (key, value) in messages_index.items():
        # check if value is a dm then add to new dict
        x = dm_check(key, value)
        if x == 0:
            dms_index[key] = value
        elif x == 1:
            server_channel_index[key] = value
        else:
            null_index[key] = value

In [12]:
dms_index["444275524508844052"]

'Direct Message with Art of Marc#0007'

In [13]:
server_channel_index["950435368061251595"]

'makan'

In [14]:
null_index["819752706762997791"]

In [15]:
server_channel_message_count = 0
thread_message_count = 0
dms_message_count = 0
announcements_message_count = 0
group_dms_message_count = 0
vc_chat_message_count = 0
announcement_thread_message_count = 0
others_length = 0
for (key, value) in messages_index.items():
    # create file path
    file_path_head = "package/messages/c" + key
    csv_file_path = file_path_head + "/messages.csv"
    json_file_path = file_path_head + "/channel.json"
    
    # read json file
    with open(json_file_path, "r") as read_file:
        json_file = json.load(read_file)
        type = json_file['type']
     
    # read csv file
    with open(csv_file_path, "rb") as read_file:
        messages_file = pd.read_csv(read_file)
        messages_file.dropna(subset='Contents', inplace=True)
        msg_count = len(messages_file)
        
    if type == 0:
        # server channel
        server_channel_message_count = server_channel_message_count + msg_count
    elif type == 11:
        # thread
        thread_message_count = thread_message_count + msg_count
    elif type == 1:
        # dm
        dms_message_count = dms_message_count + msg_count
    elif type == 5:
        # announcements
        announcements_message_count = announcements_message_count + msg_count
    elif type == 3:
        # group dms
        group_dms_message_count = group_dms_message_count + msg_count
    elif type == 2:
        # vc chat
        vc_chat_message_count = vc_chat_message_count + msg_count
    elif type == 10:
        # annoucement thread
        announcement_thread_message_count = announcement_thread_message_count + msg_count
    else:
        # others
        print(type)
        print(key)
        others_length = others_length + msg_count

In [16]:
# check staff-pantry
msg = pd.read_csv("package/messages/c953865732322644038/messages.csv")
msg.dropna(subset='Contents', inplace=True)
len(msg)
# 1839

1838

In [17]:
# check general Eat Shit
msg = pd.read_csv("package/messages/c772081394427101195/messages.csv")
msg.dropna(subset='Contents', inplace=True)
len(msg)
# 1556

1556

In [18]:
# check general E404
msg = pd.read_csv("package/messages/c748565567421808741/messages.csv")
msg.dropna(subset='Contents', inplace=True)
len(msg)
# 1341

1341

In [19]:
# check website-team
msg = pd.read_csv("package/messages/c947181732807983155/messages.csv")
msg.dropna(subset='Contents', inplace=True)
len(msg)
# 1112

1112